In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pickle
import json
import random
import numpy as np
import pandas as pd
from import_casa import casa
# http://140.112.147.132:9093/api/project/export?format=JSON

In [5]:
from datetime import date
today = date.today()
dir_name = today.strftime("%Y%m%d")
port_dict = {
    9091:'Andrea',
    9092: '品而',
    9093: 'Amy',
    9094: 'Nana'}

In [6]:
TARGET_DIR = dir_name
annots = []
# curl http://localhost:8080/api/export?format=JSON > exported_results.tar.gz
for port in range(9091, 9094+1):
    annot_path = str(casa.get_data_path() / f"annot_data/annotated_data_bkup/{TARGET_DIR}/result_{{}}.json")
    with open(annot_path.format(port), "r", encoding="UTF-8") as fin:
        annots.append(json.load(fin))

In [18]:
aa = casa.get_data_path() / f"annot_data/annotated_data_bkup/{TARGET_DIR}"
[x for x in aa.iterdir() if x.suffix==".json"]

[WindowsPath('E:/LangOn/casa/data/annot_data/annotated_data_bkup/20210320/result_9091.json'),
 WindowsPath('E:/LangOn/casa/data/annot_data/annotated_data_bkup/20210320/result_9092.json'),
 WindowsPath('E:/LangOn/casa/data/annot_data/annotated_data_bkup/20210320/result_9093.json'),
 WindowsPath('E:/LangOn/casa/data/annot_data/annotated_data_bkup/20210320/result_9094.json')]

In [20]:
dir_path = casa.get_data_path() / "annot_data/annotated_data_bkup"
chdirs = sorted([x for x in dir_path.iterdir() if x.is_dir])

In [22]:
chdirs[-1].name

'20210320'

In [78]:
annot = annots[0]

In [79]:
list(annot[0].keys())

['completions', 'data', 'id']

In [86]:
len(annots[0])

50

In [87]:
annot[0]["completions"]

[{'created_at': 1616117562,
  'id': 43001,
  'lead_time': 41.751,
  'result': [{'from_name': 'aspect-label',
    'id': '4lW7__ykGD',
    'to_name': 'text',
    'type': 'hypertextlabels',
    'value': {'end': '/div[1]/ol[1]/li[1]/text()[1]',
     'endOffset': 28,
     'htmllabels': ['Context'],
     'start': '/div[1]/ol[1]/li[1]/text()[1]',
     'startOffset': 4,
     'text': '台星的態度就是在等宿主台哥，逸以待勞，準備寄生。'}},
   {'from_name': 'ctx-Entity',
    'id': '4lW7__ykGD',
    'to_name': 'text',
    'type': 'choices',
    'value': {'choices': ['臺灣之星'],
     'end': '/div[1]/ol[1]/li[1]/text()[1]',
     'endOffset': 28,
     'start': '/div[1]/ol[1]/li[1]/text()[1]',
     'startOffset': 4,
     'text': '台星的態度就是在等宿主台哥，逸以待勞，準備寄生。'}},
   {'from_name': 'ctx-Attribute',
    'id': '4lW7__ykGD',
    'to_name': 'text',
    'type': 'choices',
    'value': {'choices': ['[通訊]頻段'],
     'end': '/div[1]/ol[1]/li[1]/text()[1]',
     'endOffset': 28,
     'start': '/div[1]/ol[1]/li[1]/text()[1]',
     'startOffset': 4

In [46]:
annot[0]["id"]

43

In [59]:
def extract_result(annot_item, port):
    serial = annot_item["data"]["serial"]
    thread_idx = annot_item["data"]["thread_idx"]
    result = annot_item["completions"][0]["result"]
    
    entries = []
    for x in result:
        from_name = x.get("from_name", "")
        ent_type = x["type"]
        value_dict = x.get("value", {})
        ent_choice = value_dict.get("choices", [""])[0]
        ent_rating = value_dict.get("rating", -1)
        ent_text = value_dict.get("text", "")
        ent_value = ent_rating if ent_type == "rating" else ent_choice
        if isinstance(ent_text, list):
            ent_text = "".join(ent_text)
        entries.append((
            port,
            thread_idx, serial,
            from_name, ent_type, ent_value, ent_text
        ))
        
    return entries

In [60]:
extract_result(annot[0], 9091)

[(9091,
  3260,
  43,
  'aspect-label',
  'hypertextlabels',
  '',
  '台星的態度就是在等宿主台哥，逸以待勞，準備寄生。'),
 (9091, 3260, 43, 'ctx-Entity', 'choices', '臺灣之星', '台星的態度就是在等宿主台哥，逸以待勞，準備寄生。'),
 (9091,
  3260,
  43,
  'ctx-Attribute',
  'choices',
  '[通訊]頻段',
  '台星的態度就是在等宿主台哥，逸以待勞，準備寄生。'),
 (9091, 3260, 43, 'ctx-sentiment', 'rating', '', '台星的態度就是在等宿主台哥，逸以待勞，準備寄生。'),
 (9091, 3260, 43, 'ctx-note', 'textarea', '', '缺 attribute'),
 (9091, 3260, 43, 'Relevance', 'choices', '主文包含評價性敘述', '')]

In [92]:
colnames = "port,thread_idx,serial,from_name,ent_type,ent_choice,ent_text".split(",")
def convert_annot_result(annot_obj, port):
    entries = []
    for annot_x in annot_obj:
        entries.extend(extract_result(annot_x, port))
    return pd.DataFrame(entries, columns=colnames)

In [93]:
dframe

,port,thread_idx,serial,from_name,ent_type,ent_choice,ent_text
0,9094,3260,43,aspect-label,hypertextlabels,,台星的態度就是在等宿主台哥，逸以待勞，準備寄生。
1,9094,3260,43,ctx-Entity,choices,臺灣之星,台星的態度就是在等宿主台哥，逸以待勞，準備寄生。
2,9094,3260,43,ctx-Attribute,choices,[通訊]頻段,台星的態度就是在等宿主台哥，逸以待勞，準備寄生。
3,9094,3260,43,ctx-sentiment,rating,,台星的態度就是在等宿主台哥，逸以待勞，準備寄生。
4,9094,3260,43,ctx-note,textarea,,缺 attribute
...,...,...,...,...,...,...,...
358,9094,315,8,attr-norm,choices,[通訊]頻段,5G
359,9094,315,8,aspect-label,hypertextlabels,,不靠譜
360,9094,315,8,sentiment,rating,,不靠譜
361,9094,315,8,,relation,,


In [94]:
for idx, port in enumerate(range(9091, 9094+1)):
    dframe = convert_annot_result(annots[idx], port)
    dframe.to_csv(casa.get_data_path() / f"annot_lstudio/{TARGET_DIR}/annot_frame_{port}.csv")

In [97]:
text = """
&lt;div class='thread-wrapper' id='D2020010117472095448797207654410081352349840177968''&gt;
&lt;span class='source'&gt;3. [Ptt] &lt;/span&gt;&lt;span class='timestamp'&gt;Wed Jan  1 17:47:20 2020&lt;/span&gt;
&lt;div class='title'&gt;Fw: [情報] HamiPay x中華電信聯名卡 超商10%感應3%&lt;/div&gt;
&lt;div class='main-text'&gt;
作者: tzunghan () 看板: MobilePay標題: [情報] HamiPay x中華電信聯名卡
超商10%感應3%時間: Wed Jan  1 14:50:04
2020有新增一些限制條款喔然後要登錄2020/1/1-2020/03/31超商使用HAMIPAY 10%
(2000名)使用HAMIPAY感應支付 3%限時加碼活動注意事項
活動一「超商10%」：四大超商包含7-ELEVEN、全家便利商店、萊爾富與OK超商，限以Hami
Pay感應支付消費，且該卡，活動期間活動二「Hami Pay 3%」：需以Hami
Pay感應支付消費始符合回饋資格，其中1%為限時活動加碼回饋，；其餘2%為中華電信聯名卡Hami
Point回饋計畫中之1%基本回饋+1%指定通路加碼回饋，回饋時間請參照Hami Point回饋計畫說明。
活動三「一般消費2%」：始符合回饋資格，；其餘1%為中華電信聯名卡Hami
Point回饋計畫中之基本回饋，回饋時間請參照Hami Point回饋計畫說明。    上述限時活動加碼回饋之Hami
Point將於活動結束後統一結算並回饋至持卡人的中華電信Hami Point會員帳戶，預計於2020/6/30前完成回饋
，限量回饋將依成功登錄且優先符合活動消費門檻之優先順序作為回饋依據，如額滿將不再回饋，且不另行通知。    上述活動皆需
以中華電信聯名卡刷卡消費，消費期間須介於2020/1/1~2020/3/31，並於2020/4/20前入帳，且持卡人歸戶
之回饋方案須於活動期間起選擇為Hami
Point回饋計畫才符合回饋資格，且排除中國信託共同不回饋項目(活動一之超商消費除外)。--&lt;/div&gt;

&lt;ol class='reply-wrap'&gt;
  &lt;li class='reply-text' id='reply-0'&gt; 新增消費5000門檻就bye了  &lt;/li&gt;
  &lt;li class='reply-text' id='reply-1'&gt; 中華電信KOKO卡？  &lt;/li&gt;
  &lt;li class='reply-text' id='reply-2'&gt; 5千，還限2千名 pass  &lt;/li&gt;
  &lt;li class='reply-text' id='reply-3'&gt; pass  &lt;/li&gt;
  &lt;li class='reply-text' id='reply-4'&gt; 5000 超有事
   廢牡蠣真的對自己很有信心 XDDDD  &lt;/li&gt;
  &lt;li class='reply-text' id='reply-5'&gt; 留著墊桌腳  &lt;/li&gt;
  &lt;li class='reply-text' id='reply-6'&gt; 活動二沒有門檻，3%還不錯啊！  &lt;/li&gt;
  &lt;li class='reply-text' id='reply-7'&gt; 活動二有個 1%指定通路加碼回饋,
   沒在指定通路消費的話就 沒那 1% 了吧 不過直接用 Hami Pay感應支付就可以了  &lt;/li&gt;
  &lt;li class='reply-text' id='reply-8'&gt; 電信費還有3%嗎？ 現在還有誰3% QQ  &lt;/li&gt;
  &lt;li class='reply-text' id='reply-9'&gt; 限量2000名  &lt;/li&gt;
  &lt;li class='reply-text' id='reply-10'&gt;
   有點複雜。一般消費不知道有沒有算中華電信自扣0.0？  &lt;/li&gt;
  &lt;li class='reply-text' id='reply-11'&gt;
   1呵呵，2新光10%玉山8%繳費用完就輪到了，3呵呵  &lt;/li&gt;
  &lt;li class='reply-text' id='reply-12'&gt; 下去嗎，今年還是玉山獨秀  &lt;/li&gt;
&lt;/ol&gt;
&lt;/div&gt;
"""

In [100]:
import html
print(html.unescape(text))


<div class='thread-wrapper' id='D2020010117472095448797207654410081352349840177968''>
<span class='source'>3. [Ptt] </span><span class='timestamp'>Wed Jan  1 17:47:20 2020</span>
<div class='title'>Fw: [情報] HamiPay x中華電信聯名卡 超商10%感應3%</div>
<div class='main-text'>
作者: tzunghan () 看板: MobilePay標題: [情報] HamiPay x中華電信聯名卡
超商10%感應3%時間: Wed Jan  1 14:50:04
2020有新增一些限制條款喔然後要登錄2020/1/1-2020/03/31超商使用HAMIPAY 10%
(2000名)使用HAMIPAY感應支付 3%限時加碼活動注意事項
活動一「超商10%」：四大超商包含7-ELEVEN、全家便利商店、萊爾富與OK超商，限以Hami
Pay感應支付消費，且該卡，活動期間活動二「Hami Pay 3%」：需以Hami
Pay感應支付消費始符合回饋資格，其中1%為限時活動加碼回饋，；其餘2%為中華電信聯名卡Hami
Point回饋計畫中之1%基本回饋+1%指定通路加碼回饋，回饋時間請參照Hami Point回饋計畫說明。
活動三「一般消費2%」：始符合回饋資格，；其餘1%為中華電信聯名卡Hami
Point回饋計畫中之基本回饋，回饋時間請參照Hami Point回饋計畫說明。    上述限時活動加碼回饋之Hami
Point將於活動結束後統一結算並回饋至持卡人的中華電信Hami Point會員帳戶，預計於2020/6/30前完成回饋
，限量回饋將依成功登錄且優先符合活動消費門檻之優先順序作為回饋依據，如額滿將不再回饋，且不另行通知。    上述活動皆需
以中華電信聯名卡刷卡消費，消費期間須介於2020/1/1~2020/3/31，並於2020/4/20前入帳，且持卡人歸戶
之回饋方案須於活動期間起選擇為Hami
Point回饋計畫才符合回饋資格，且排除中國信託共同不回饋項目(活動一之超商消費除外)。--</div>

<ol class=